In [4]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [ ]:
# sequential 连续的
# synthetic data合成的数据

In [50]:
# 生成数据集
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [6]:
"""
PyTorch提供了data包来读取数据
"""

# 读取数据
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)


In [7]:
# 读取并打印第一个小批量数据样本
for X, y in data_iter:
    print(X, y)
    break


tensor([[ 0.2873,  1.5844],
        [ 1.8630, -0.9623],
        [-1.5222,  0.6606],
        [-2.1451,  0.6497],
        [ 0.5533,  0.8830],
        [-1.1515,  0.5055],
        [-0.6753,  0.5404],
        [-1.0459, -1.4009],
        [ 0.4299, -1.3397],
        [-0.0290, -0.4355]]) tensor([-0.6151, 11.1975, -1.0948, -2.3116,  2.2915,  0.2048,  1.0291,  6.8610,
         9.6093,  5.6249])


In [17]:

"""
在上一节从零开始的实现中，我们需要定义模型参数，并使用它们一步步描述
模型时怎样计算的。当模型结构变得更复杂时， 这些步骤就变得更繁琐。 
其实， PyTorch提供了大量预定义的层，这使我们只需关注使用哪些层来构造
模型。 下面将介绍如何使用PyTorch更简洁地定义线性回归。
首先，导入torch.nn模块，实际上，'nn'是neural networks(神经网络)
的缩写。 顾名思义， 该模块定义了大量神经网络的层，之前我们已经用过了
autograd, 而nn就是利用autograd来定义模型，nn的核心数据结构是module,
它是一个抽象概念，既可以表示神经网络中的某个层(layer), 也可以表示
一个包含很多层的神经网络。在实际使用中，最常用的做法是继承nn.module,
撰写自己的网络层。一个nn.module实例应该包含一些层以及返回输出的前向传播
（forward）方法。 下面先来看看如何用nn.module实现一个线性回归模型
"""
import torch.nn as nn

# =======1. 定义模型==================
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构




LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [9]:
"""
我们还可以用nn.Sequential来更加方便地搭建网络， Sequential是一个有序的容器，
网络层将按照在传入Sequential的顺序依次被添加到计算图中。
"""
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其它层
    )

In [23]:
# 写法二：
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))

In [38]:
# 写法三：
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
    ('linear', nn.Linear(num_inputs, 1))
    # ...
]))
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [39]:
# 可以通过net.parameters()来查看模型所有的学习参数，此函数将返回一个生成器
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.5763, -0.2825]], requires_grad=True)
Parameter containing:
tensor([-0.2649], requires_grad=True)


In [40]:
"""
线性回归在神经网络图中的表示，作为一个单层神经网络， 线性回归输出层中的神经元
和输入层中各个输入完全连接。因此，线性回归的输出层又叫做全连接层
"""

"""
注意：torch.nn仅支持输入一个batch的样本不支持单个样本输入； 如果只有单个样本，
可以使用input.unsqueeze(0)来添加一维
"""

'\n注意：torch.nn仅支持输入一个batch的样本不支持单个样本输入； 如果只有单个样本，\n可以使用input.unsqueeze(0)来添加一维\n'

In [41]:
# =======2. 初始化模型参数==================
"""
在使用net前， 我们需要初始化模型采纳数，如线性回归模型中的权重和偏差。 PyTorch
在init模块中提供了多种参数初始化的方法。 这里的init是initializer的缩写形式。
我们通过init.normal_将权重参数每个元素初始化为随机采样于均值为0，标准差为
0.01的正态分布。 偏差会初始化为0
"""
from torch.nn import init
init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0) # 也可以：net[0].bias.data.fill_(0)

# from torch.nn import init
# init.normal_(net.linear.weight, mean=0, std=0.01)
# init.constant_(net.linear.bias, val=0) # 也可以：net[0].bias.data.fill_(0)


"""
如果net是用类定义模型的那个代码自定义的，那么上面的代码就会报错，net[0].weight
应改为net.linear.weight, bias亦然，因为net[0]这样根据下表访问子模块的写法只有
当net是ModuleList或者Sequential实例时才可以。
"""


'\n如果net是用类定义模型的那个代码自定义的，那么上面的代码就会报错，net[0].weight\n应改为net.linear.weight, bias亦然，因为net[0]这样根据下表访问子模块的写法只有\n当net是ModuleList或者Sequential实例时才可以。\n'

In [42]:
# =======3. 定义损失函数=================
"""
PyTorch在nn模块中提供了各种损失函数，这些损失函数可看作是一种特殊的层。PyTorch
也将这些损失函数实现为nn.Module的子类。 我们现在使用它提供的均方误差损失作为
模型的损失函数"""

loss = nn.MSELoss()

In [44]:
# =======4. 定义优化算法=================
"""
同样，我们也无序自己实现小批量随机梯度下降算法，torch.optim模块提供了很多常用
的优化算法比如SGD, Adam和RMSProp等。 下面我们创建一个用于优化net所有参数的
优化器实例，并指定学习率为0.03的小批量随机梯度下降(SGD)为优化算法
"""
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)



SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [43]:
"""
我们还可以为不同子网络设置不同的学习率，这在finetune经常用到。 例：
"""
optimizer =optim.SGD([
                # 如果对某个参数不指定学习率，就使用最外层的默认学习率
                {'params': net.subnet1.parameters()}, # lr=0.03
                {'params': net.subnet2.parameters(), 'lr': 0.01}
            ], lr=0.03)


AttributeError: 'Sequential' object has no attribute 'subnet1'

In [45]:
"""
有时候我们不想让学习率固定成一个常数， 那如何调整学习率呢？ 主要有两种做法，一种是修改
optimizer.param_groups中对应的学习率，另一种是更简单也是较为推荐的做法--新建优化器，
由于optimizer十分轻量级，构建开销很小，故而可以构建新的optimizer， 但是后者对于使用
动量的优化器(如Adam), 会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况
"""

# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1  # 学习率为之前的0.1倍

In [47]:
# =======5. 训练模型=================
"""在使用Gluon训练模型时，我们通过调用optim实例的step函数来迭代模型参数。按照小批量随机梯度
下降的定义。我们在step函数中指明批量大小，从而对批量中样本梯度求平均"""

num_epochs = 3
for epoch in range(1, num_epochs+1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1,1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
        
    print('epoch %d, loss: %f' % (epoch, l.item()))


epoch 1, loss: 8.793560
epoch 2, loss: 2.885707
epoch 3, loss: 0.572145


In [48]:
"""
下面我们分别比较学到的模型采纳数和真实的模型参数，我们从net获得需要的层，并访问其权重(weight)
和偏差(bias). 学到的参数和真实的参数很接近

"""

dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)



[2, -3.4] Parameter containing:
tensor([[ 1.7728, -2.9196]], requires_grad=True)
4.2 Parameter containing:
tensor([3.5580], requires_grad=True)


In [ ]:
"""
小结：
1. 使用PyTorch可以更简洁地实现模型
2. torch.utils.data模块提供了有关数据处理的工具，torch.nn模块定义了大量神经网络的
层。 torch.nn.init模块定义了各种初始化方法，torch.optim模块提供了很多常用的优化算法。

"""